In [1]:
import pandas as pd
import wrds
import numpy as np
from DataHandler import DataHandler
from pandas.tseries.offsets import *

In [2]:
db = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [3]:
# ccm_query = f"""
#             SELECT gvkey, lpermno as permno, linktype, linkprim, 
#             linkdt, linkenddt
#             FROM crsp.ccmxpf_linktable
#             WHERE substr(linktype,1,1)='L'
#             AND (linkprim ='C' or linkprim='P')
#         """

# ccm = db.raw_sql(ccm_query)

In [4]:
funda = pd.read_csv('input_data/funda.csv', index_col=0)
crsp = pd.read_csv('input_data/crsp.csv', index_col=0)
ccm = pd.read_csv('input_data/ccm.csv', index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/eric/Documents/GitHub/TheBot/input_data/funda.csv'

In [ ]:

crsp = crsp.dropna(subset=['prc'])
crsp['me'] = crsp['prc'].abs() * crsp['shrout']  # calculate market equity

# if Market Equity is Nan then let return equals to 0
crsp['ret'] = np.where(crsp['me'].isnull(), 0, crsp['ret'])

# impute me
crsp = crsp.sort_values(by=['permno', 'date']).drop_duplicates()
crsp['me'] = np.where(crsp['permno'] == crsp['permno'].shift(1), crsp['me'].fillna(method='ffill'), crsp['me'])


In [ ]:
ccm['linkdt'] = pd.to_datetime(ccm['linkdt'])
ccm['linkenddt'] = pd.to_datetime(ccm['linkenddt'])
funda['datadate'] = pd.to_datetime(funda['datadate'])
crsp['date'] = pd.to_datetime(crsp['date'])

# if linkenddt is missing then set to today date
ccm['linkenddt'] = ccm['linkenddt'].fillna(pd.to_datetime('today'))

# merge ccm and comp
ccm1 = pd.merge(funda, ccm, how='left', on=['gvkey'])

# we can only get the accounting data after the firm public their report
# for annual data, we use 4, 5 or 6 months lagged data, now we follow Hou, Xue and Zhang (2015) use 4 months lag
ccm1['yearend'] = ccm1['datadate'] + YearEnd(0)
ccm1['jdate'] = ccm1['datadate'] + MonthEnd(4)

# set link date bounds
ccm2 = ccm1[(ccm1['jdate'] >= ccm1['linkdt']) & (ccm1['jdate'] <= ccm1['linkenddt'])]

ccm2['permno'] = ccm2['permno'].astype(int)

In [ ]:
def merge_funda_with_crsp(ccm2, crsp):
    """Merge funda with crsp to get market cap on the closest date for each datadate in funda."""
    ccm2['jdate'] = pd.to_datetime(ccm2['jdate'])
    crsp['date'] = pd.to_datetime(crsp['date'])

    # Merge on 'cusip' and closest date prior to or on 'datadate'.
    # Sometimes 'datadate' is on the weekend so this is necessary.
    merged_funda = pd.merge_asof(
        ccm2.sort_values('jdate'),
        crsp[['permno', 'date', 'me']].sort_values('date'),
        left_on='jdate',
        right_on='date',
        by='permno',
        direction='backward'
    )
    return merged_funda

In [ ]:
merged_data = merge_funda_with_crsp(ccm2, crsp)